In [ ]:
import os
import os.path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from evaluation_base import linearRegression
from IPython.display import display
import ipywidgets as widgets

plt.rcParams['figure.figsize'] = 10, 6


def H1(text):
    return widgets.HTML(f"<h1>{text}</h1>")
    
def H2(text):
    return widgets.HTML(f"<h2>{text}</h2>")

def evaluate_stat(data_dir):
    display(H2("Teil 1: Statische Messung"))
    
    # load data
    data_part1 = np.load(os.path.join(data_dir, "DataV2Part1.npy"))
    df = pd.DataFrame(data_part1.T, columns=["m", "l"])
    df["x"] = df.l - df.l[0]
    display(df)

    slope, intercept = linearRegression(df.m, df.l, "Masses [g]", "Length [mm]")

    print("Slope b = {0} m/kg".format(round(slope, 2)))
    print("Spring constant D = g / b = {0} kg/s^2".format(round(9.81 / slope, 2)))

def evaluate_dyn(data_dir):
    display(H2("Teil 2: Dynamische Messung"))
        
    # load data
    data_part2 = np.load(os.path.join(data_dir, "DataV2Part2.npy"))
    df = pd.DataFrame(data_part2.T, columns=["m", "t"])
    
    # Divided by 10, because the data saves 10 Period times
    df["t_squared"] = (df.t / 10)**2
    df["t_squared_rounded"] = np.around(df.t_squared, 3)

    display(df)
    
    slope, intercept = linearRegression(df.m, df.t_squared, "Masses [g]", "Period times squared [s^2]")

    print("Slope b = {0} kg/s^2".format(round(slope, 3) * 1000))
    print("Spring constant D = 4π^2 / b = {0} kg/s^2".format(round(4 * np.pi**2 / slope / 1000, 2)))

    display(H2("Teil 3: Bestimmung der Federmasse"))
    
    mz = -intercept / slope
    mf = - 3 * mz
    print("Spring mass m_F = {:.2f} g".format(mf))

# Interactive student selection
with open("students.txt", "r") as f:
    students = [line.strip("\n") for line in f if not line.startswith("#")]
    
tb = widgets.ToggleButtons(
    options=students,
    description='Name:',
)
display(tb)

output = widgets.Output()
display(output)

@output.capture(clear_output=True)
def on_student_change(change):
    name = change["new"]
    display(H1(f"Auswertung für: {name}"))

    name_escaped = name.replace(" ", "_")
    data_dir = os.path.join("V2", name_escaped)
    
    evaluate_stat(data_dir)
    evaluate_dyn(data_dir)

tb.observe(on_student_change, names="value")
on_student_change({"new": students[0]})